# Torch bridges examples

## Homogeneous dataset

In [9]:
import copy
import pickle

import numpy as np
from datasets import load_dataset
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)
from plaid.containers.sample import Sample
from plaid_ops.mesh.feature_engineering import update_dataset_with_sdf
from plaid_ops.mesh.transformations import (
    compute_bounding_box,
    project_on_regular_grid,
)
from torch.utils.data import DataLoader

from plaid_bridges.torch.base import HomogeneousDataset
from plaid_bridges.torch.grid import GridFieldsAndScalarsDataset

In [10]:
hf_dataset = load_dataset("PLAID-datasets/VKI-LS59", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train")[:10]

dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

Converting Hugging Face dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 56.02it/s]


In [11]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [
    f for f in all_feat_ids if "field" in f.values() and "Base_2_2" in f.values()
]

in_feature_identifiers = scalar_features
out_feature_identifiers = field_features

print(in_feature_identifiers)
print(out_feature_identifiers)

Dataset(10 samples, 8 scalars, 0 time_series, 8 fields)
[{'type': 'scalar', 'name': 'Pr'}, {'type': 'scalar', 'name': 'Q'}, {'type': 'scalar', 'name': 'Tr'}, {'type': 'scalar', 'name': 'angle_in'}, {'type': 'scalar', 'name': 'angle_out'}, {'type': 'scalar', 'name': 'eth_is'}, {'type': 'scalar', 'name': 'mach_out'}, {'type': 'scalar', 'name': 'power'}]
[{'type': 'field', 'name': 'rov', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'ro', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'mach', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'roe', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'nut', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': 0.0}, {'type': 'field', 'name': 'sdf', 'base_name': 'Base_2_2', 'zone_name':

In [12]:
homogen_dataset = HomogeneousDataset(
    dataset=dataset,
    in_feature_identifiers=in_feature_identifiers,
    out_feature_identifiers=out_feature_identifiers,
)

homogen_dataset.show_details()

RegressionDataset (10 sample, 8 input features, 7) output features)
Input features : ['Pr (scalar)', 'Q (scalar)', 'Tr (scalar)', 'angle_in (scalar)', 'angle_out (scalar)', 'eth_is (scalar)', 'mach_out (scalar)', 'power (scalar)']
Output features: ['rov (field)', 'ro (field)', 'mach (field)', 'roe (field)', 'nut (field)', 'sdf (field)', 'rou (field)']


In [13]:
loader = DataLoader(
    homogen_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = out_feature_identifiers[0]
before = copy.deepcopy(dataset[ids[1]].get_feature_from_identifier(out_feat_id))

predictions = []
for batch in loader:
    for torch_sample in batch:
        predictions.append(torch_sample[1])

dataset_pred = homogen_dataset.inverse_transform(predictions)

after = copy.deepcopy(dataset_pred[ids[1]].get_feature_from_identifier(out_feat_id))

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))

Error after transform then inverse transform (2nd sample):
0.0


## Projection on constant rectilinear grid, with scalars as constant fields

In [14]:
hf_dataset = load_dataset(
    "PLAID-datasets/2D_Multiscale_Hyperelasticity", split="all_samples"
)
pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids_train = pb_def.get_split("DOE_train")[:10]

dataset_train, _ = huggingface_dataset_to_plaid(
    hf_dataset, ids=ids_train, processes_number=5
)
n_train = len(dataset_train)
print(dataset_train)

Converting Hugging Face dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 178.66it/s]

Dataset(10 samples, 4 scalars, 0 time_series, 7 fields)


In [15]:
dims = (101, 101)
dataset_train = update_dataset_with_sdf(dataset_train, verbose=True)

print(dataset_train.get_sample_ids())

bbox = compute_bounding_box(dataset_train)
projected_dataset_train = project_on_regular_grid(
    dataset_train, dimensions=dims, bbox=bbox, verbose=True
)

all_feat_ids = dataset_train[ids_train[0]].get_all_features_identifiers()
scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

100%|██████████| 10/10 [00:00<00:00, 15.33it/s]


[376, 377, 378, 379, 380, 381, 382, 383, 384, 385]


100%|██████████| 10/10 [00:00<00:00, 32.46it/s]


In [16]:
torch_dataset = GridFieldsAndScalarsDataset(
    dataset=projected_dataset_train,
    dimensions=dims,
    in_feature_identifiers=[field_features[0], scalar_features[0]],
    out_feature_identifiers=[scalar_features[1], field_features[1]],
)

loader = DataLoader(
    torch_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = scalar_features[0]

before = copy.deepcopy(
    projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

predictions = []
for batch in loader:
    for torch_sample in batch[1]:
        predictions.append(torch_sample.detach().cpu().numpy())

projected_dataset_train = torch_dataset.inverse_transform(predictions)

after = copy.deepcopy(
    projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))

Error after transform then inverse transform (2nd sample):
0.0
